# dev

In [2]:
# constants
bucket = "k9-kedro-on-aws"
tbl = "transactions"

## Check AWS conf

Setup the credentials in your preferred manner.

In [3]:
%%bash
aws sts get-caller-identity

{
    "UserId": "AIDARAMT7QKYJ6DR7ITQU",
    "Account": "069566694064",
    "Arn": "arn:aws:iam::069566694064:user/k9"
}


In [4]:
%%bash -s $bucket
aws s3 ls $1

                           PRE athena-query-outputs/
                           PRE parsed/
                           PRE transactions/


## Check S3 connection on pyspark

Note that we will need two additional `jars` in order to access AWS S3 from Spark:

- [hadoop-aws](https://mvnrepository.com/artifact/org.apache.hadoop/hadoop-aws)
- [aws-java-sdk-bundle](https://mvnrepository.com/artifact/com.amazonaws/aws-java-sdk-bundle)

Make sure they are present in your `SPARK_HOME` jars folder.
In the case of a `pip install pyspark`, the folder should be something like `<your-python-lib-path>/site-packages/pyspark/jars`.

In [5]:
from pyspark.sql import SparkSession

# spark configuration is handled by conf/base/spark.yml
# refer to the src/hooks.py for details
spark = SparkSession.builder.getOrCreate()

In [6]:
spark.sparkContext.getConf().getAll()

[
    (
        'spark.driver.extraJavaOptions',
        '-XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED'
    ),
    ('spark.sql.warehouse.dir', 'file:/Users/kyle_chung/kedro-on-aws/notebooks/spark-warehouse'),
    ('spark.scheduler.mode', 'FAIR'),
    ('spark.executor.id', 'driver'),
    ('spark.app.name', 'iris'),
    ('spark.hadoop.fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem'),
    ('spark.app.startTime', '1673251548378'),
    ('spark.app.submitTime', '1673251548203'),
    ('spark.sql.catalogImplementation', 'hive'),
    ('spark.sql.execution.arrow.pyspark.enabled', 'True'),
    ('spark.rdd.compress', 'True'),
    ('spark.app.id', 'local-1673251549318'),
    (
        'spark.executor.extraJavaOptions',
        '-XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED'
    ),
    ('spark.serializer.objectStreamReset', '100'),
    ('spark.driver.host', '172.30.80.61'),
    ('spark.submit.pyFiles', ''),
    ('spark.submit.deployMode', 'client'),
    ('spark.driver.maxResultSize', '3g'),
    ('spark.ui.showConsoleProgress', 'true'),
    ('spark.master', 'local[2]'),
    ('spark.driver.port', '60926')
]

In [7]:
spark.read.parquet(f"s3a://{bucket}/{tbl}").printSchema()

23/01/09 16:06:13 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


23/01/09 16:06:16 WARN DataSource: Found duplicate column(s) in the data schema and the partition schema: `date`
root
 |-- gas: long (nullable = true)
 |-- hash: string (nullable = true)
 |-- input: string (nullable = true)
 |-- nonce: long (nullable = true)
 |-- value: double (nullable = true)
 |-- block_number: long (nullable = true)
 |-- block_hash: string (nullable = true)
 |-- transaction_index: long (nullable = true)
 |-- from_address: string (nullable = true)
 |-- to_address: string (nullable = true)
 |-- gas_price: long (nullable = true)
 |-- receipt_cumulative_gas_used: long (nullable = true)
 |-- receipt_gas_used: long (nullable = true)
 |-- receipt_contract_address: string (nullable = true)
 |-- receipt_status: long (nullable = true)
 |-- receipt_effective_gas_price: long (nullable = true)
 |-- transaction_type: long (nullable = true)
 |-- max_fee_per_gas: long (nullable = true)
 |-- max_priority_fee_per_gas: long (nullable = true)
 |-- block_timestamp: timestamp (nullable =

## Test kedro catalog

In [8]:
catalog.list()

[
    'transactions',
    'parsed',
    'parameters',
    'params:train_fraction',
    'params:random_state',
    'params:target_column'
]

In [9]:
transactions = catalog.load("transactions")

[01/09/23 16:06:18] INFO     Loading data from 'transactions' (AthenaQueryDataSet)...           ]8;id=228664;file:///Users/kyle_chung/.pyenv/versions/3.9.15/envs/kedro-on-aws/lib/python3.9/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=392701;file:///Users/kyle_chung/.pyenv/versions/3.9.15/envs/kedro-on-aws/lib/python3.9/site-packages/kedro/io/data_catalog.py#343\343]8;;\

                    INFO     Attempt to UNLOAD files to                                       ]8;id=722863;file:///Users/kyle_chung/kedro-on-aws/src/iris/extras/datasets/athena_dataset.py\athena_dataset.py]8;;\:]8;id=728116;file:///Users/kyle_chung/kedro-on-aws/src/iris/extras/datasets/athena_dataset.py#105\105]8;;\
                             s3://k9-kedro-on-aws/athena-query-outputs/temp_tbl_1d109d5918                         
                             with Athena.                                                                          

[01/09/23 16:06:23] INFO     Athena query output files saved to:                              ]8;id=904742;file:///Users/kyle_chung/kedro-on-aws/src/iris/extras/datasets/athena_dataset.py\athena_dataset.py]8;;\:]8;id=279836;file:///Users/kyle_chung/kedro-on-aws/src/iris/extras/datasets/athena_dataset.py#137\137]8;;\
                             s3://k9-kedro-on-aws/athena-query-outputs/temp_tbl_1d109d5918                         

In [10]:
transactions.printSchema()

root
 |-- date: string (nullable = true)
 |-- hash: string (nullable = true)
 |-- nonce: long (nullable = true)
 |-- transaction_index: long (nullable = true)
 |-- from_address: string (nullable = true)
 |-- to_address: string (nullable = true)
 |-- value: double (nullable = true)
 |-- gas: long (nullable = true)
 |-- gas_price: long (nullable = true)
 |-- input: string (nullable = true)
 |-- receipt_cumulative_gas_used: long (nullable = true)
 |-- receipt_gas_used: long (nullable = true)
 |-- receipt_contract_address: string (nullable = true)
 |-- receipt_status: long (nullable = true)
 |-- block_timestamp: timestamp (nullable = true)
 |-- block_number: long (nullable = true)
 |-- block_hash: string (nullable = true)
 |-- max_fee_per_gas: long (nullable = true)
 |-- max_priority_fee_per_gas: long (nullable = true)
 |-- transaction_type: long (nullable = true)
 |-- receipt_effective_gas_price: long (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = t

In [18]:
parsed = catalog.load("parsed")

[01/09/23 16:07:03] INFO     Loading data from 'parsed' (SparkDataSet)...                       ]8;id=45103;file:///Users/kyle_chung/.pyenv/versions/3.9.15/envs/kedro-on-aws/lib/python3.9/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=305529;file:///Users/kyle_chung/.pyenv/versions/3.9.15/envs/kedro-on-aws/lib/python3.9/site-packages/kedro/io/data_catalog.py#343\343]8;;\

In [27]:
%%bash -s $bucket
aws s3 ls $1/parsed/ --recursive

2023-01-09 16:19:03          0 parsed/_SUCCESS
2023-01-09 16:19:02     105395 parsed/year=2023/month=1/part-00000-0d4adef1-2d6c-4881-8fe7-fce96499cfe5.c000.snappy.parquet
